# Buckling under radial force: laterally restrained spokes

In [ ]:
import bikewheellib as bl
from doetools import RadialBucklingDOE, wheel_library
from shutil import copy2
from imp import find_module

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

%matplotlib inline

In [ ]:
# Set default colors
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
                                                    '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
                                                    '#bcbd22', '#17becf'])

## Truss spokes (no spoke buckling), paired spokes

In [ ]:
truss_dir = '../data/doe/doe_rad_buckling_truss'

if True:
    doe = RadialBucklingDOE(out_dir=truss_dir,
                            opts={'spk_paired': True,
                                  'spk_eltype': 'truss',
                                  'sim_type': 'exp',
                                  'sim_u2': 'auto',
                                  'rim_perturb': [0.01e-3, 0.01e-3, 0.01e-3]})

    wheel_types = ['cheap_MTB', 'vintage_rd', 'racing_700',
                   'folding', 'track', 'tandem', 'high_wheel']

    for w_type in wheel_types:
        
        wheel = wheel_library.create_predefined_wheel(w_type)
        wheel_library.convert_to_paired(wheel)
        
        Tc, nc = bl.calc_buckling_tension(wheel)

        for Tn in [0.0, 0.10, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]:
            
            wheel.apply_tension(Tn*Tc)
            
            jobname = '{:s}_{:.2f}'.format(w_type, Tn)
            doe.add_experiment(wheel, opts={'jobname': jobname,
                                            'wheel_type': w_type,
                                            'spk_Tn': Tn,
                                            'spk_T': Tn * Tc})

    print('\nCreated {0:d} simulations'.format(len(doe.db)))

    doe.write_input_files(N_batches=4)
    doe.to_csv()
    
    # Copy postprocessing script
    copy2(src=find_module('doetools')[1] + '/postproc_rad_buckling.py',
          dst=truss_dir)

In [ ]:
# Load DOE database
print 'Loading database...'
doe_t = RadialBucklingDOE(out_dir=truss_dir, db_file=truss_dir+'/_doe_db.csv')

# Populate DOE database with results
print 'Extracting results...'
doe_t.extract_results()
print '\nDone'

In [ ]:
# Calculate lateral stiffness for each wheel
doe_t.db['K_lat'] = doe_t.db.apply(lambda x: 
                                   bl.lateral_stiffness(doe_t.wheel_from_row(x),
                                                        tension=x['spk_T']),
                                   axis=1)

# Calculate P_c, normalized by P_c(T = 0)
f = lambda r: r['Pc_nonlin'] / doe_t.db[(doe_t.db['wheel_type'] == r['wheel_type']) &
                                   (doe_t.db['spk_Tn'] == 0.0)]['Pc_nonlin'][0]

doe_t.db['Pc_norm'] = doe_t.db.apply(f, axis=1)

In [ ]:
cp = sns.color_palette('Reds', 5)

with plt.style.context('seaborn-paper'):
    fig, ax = plt.subplots(ncols=3, figsize=(6.5, 2.17))

    d = doe_t.db[(doe_t.db['wheel_type'] == 'vintage_rd') & doe_t.db['spk_Tn'].isin([0.2, 0.4, 0.6, 0.8, 0.95])]
    
    c = 0
    for i, r in d.iterrows():
        pd_data = pd.read_csv(truss_dir + '/' + r.name + '_collapse_Pd.csv')

        ax[0].plot(np.abs(1000*pd_data['U3 [m]']), np.abs(0.001*pd_data['RF2 [N]']), color=cp[c])
        c = c + 1

    ax[0].set_xlim([0., 8.])
    ax[0].set_ylim([0., 25.])
    ax[0].set_xticks([0, 2, 4, 6, 8])
    ax[0].set_yticks([0, 5, 10, 15, 20, 25])

    ax[0].set_xlabel('Lateral displacement [mm]')
    ax[0].set_ylabel('Radial load [kN]')
    
    mrk = ['o', '*', 'd', 's', '^', 'p', 'v']

    type_order = doe_t.db[['wheel_type', 'Pc_nb']].groupby('wheel_type').max().sort_values(by='Pc_nb').index

    for i, t in enumerate(type_order):
        d = doe_t.db[doe_t.db['wheel_type'] == t]
        ax[1].plot(d['rim_radius']*d['K_lat']/1000,
                   d['Pc_nonlin']/1000, mrk[i],
                   label=t.replace('_', ' '))

    ax[1].plot([0., 100.], [0., 100.], 'k--')

    ax[1].axis([0., 100., 0., 100.])
    ax[1].set_xlabel(r'\$K_{lat} R\$ [kN]')
    ax[1].set_ylabel(r'\$P_c\$ [kN]')

    ax[1].legend(loc='best')
    
    
    by_Tn = doe_t.db[['spk_Tn', 'Pc_norm']].groupby('spk_Tn')

    (_, caps, _) = ax[2].errorbar(by_Tn.mean().index,
                                  by_Tn.mean()['Pc_norm'],
                                  yerr=by_Tn.std()['Pc_norm'],
                                  fmt='o', capsize=3, label='FEA')

    for cap in caps:
        cap.set_markeredgewidth(1)

    xx = np.linspace(0, 1, 20)
    ax[2].plot([0., 1.], [1., 0.], 'k--', label='linear')
    ax[2].plot(xx, 1 - xx**2, 'k-.', label='quadratic')

    ax[2].set_xlim([0., 1.])
    ax[2].set_xlabel(r'\$T/T_c\$')
    ax[2].set_ylabel(r'\$P_c/P_{c,0}\$')
    
    
    plt.tight_layout()
    plt.savefig('../figs/buckling_ext_loads/_python_rad_buckling.pdf')